This notebook parses text from raw reddit submissions of a given subreddit and outputs it to the **outdata** directory.

In [2]:
import pandas as pd
import polars as pl

In [15]:
schema = {
    "created_utc": pl.Int32,
    "id": pl.Utf8,
    "is_self": pl.Boolean,
    "title": pl.Utf8,
    "selftext": pl.Utf8,
    "subreddit": pl.Utf8,
}
df = pl.read_ndjson("./rawdata/test.jsonl", schema=schema)

In [16]:
df = df.filter(
    pl.col("is_self") == True,
)
df = df.filter(~pl.col("selftext").is_in(["[deleted]", "[removed]"]))
df = df.filter(pl.col("created_utc") > 1640995200) # 01/01/22 00:00
df = df.sort("created_utc", descending=True)
df = df.with_columns(
    pl.col("selftext").str.replace("\\\\", "\\")
)

In [17]:
len(df)

3

In [14]:
df[19]["selftext"].item()

"It's been very tough for me. I'm going to have to let go of the car. It started with not being able to afford the insurance but now I know in order to get back on my feet and rebuild my credit which was in the 700s not even 4 months ago I need to let go of it. \\n\\nIs this possible? What methods are there to this? I will be talking to the bank on Tuesday and making an appointment but would like some advice her"

In [18]:
df.write_csv("./outdata/personalfinance_submissions_2022_TEST.csv")